## imports

In [ ]:
import pandas as pd
from scripts.data_loader import load_data
from scripts.plot_util import PlotHelper
# from scripts.preprocess import preprocess_data


load data basic eda

In [64]:
df = load_data("../data/raw_analyst_ratings.csv")
print(df.head())
print(df.columns)

   Unnamed: 0                                           headline  \
0           0            Stocks That Hit 52-Week Highs On Friday   
1           1         Stocks That Hit 52-Week Highs On Wednesday   
2           2                      71 Biggest Movers From Friday   
3           3       46 Stocks Moving In Friday's Mid-Day Session   
4           4  B of A Securities Maintains Neutral on Agilent...   

                                                 url          publisher  \
0  https://www.benzinga.com/news/20/06/16190091/s...  Benzinga Insights   
1  https://www.benzinga.com/news/20/06/16170189/s...  Benzinga Insights   
2  https://www.benzinga.com/news/20/05/16103463/7...         Lisa Levin   
3  https://www.benzinga.com/news/20/05/16095921/4...         Lisa Levin   
4  https://www.benzinga.com/news/20/05/16095304/b...         Vick Meyer   

                        date stock  
0  2020-06-05 10:30:54-04:00     A  
1  2020-06-03 10:45:20-04:00     A  
2  2020-05-26 04:30:07-04:00 

In [ ]:
print(df.info())
print(df.shape)

In [ ]:
print(type(df['headline']))
print(df['headline'].head())

In [ ]:
# --- 1. Handling Null Values ---

print("\n--- 1. Null Value Check (Before Cleaning) ---")
print(df.isnull().sum())

# We focus on the 'headline' column since it's used for sentiment analysis.
# Strategy: Drop rows where the 'headline' is null, as we can't analyze empty text.
df.dropna(subset=['headline'], inplace=True)
print("\n--- Null Value Check (After Cleaning) ---")
print(f"Remaining rows after dropping null headlines: {df.shape[0]}")

In [ ]:
# --- 2. Date Structure and Time Series Preparation  ---

if 'date' in df.columns:
    
    # 1. Force conversion using 'ISO8601' format and ensure UTC conversion.
    # Setting utc=True handles the mixed timezone offsets by normalizing all values to UTC 
    # and ensures the resulting column is a proper datetime dtype.
    df['date'] = pd.to_datetime(df['date'], format='ISO8601', utc=True)
    
    print("\n--- 2. Date Column Structure (After Conversion) ---")
    print(df['date'].head())
    print(f"Data type is now: {df['date'].dtype}")
    # 2. Set the Index
    # Note: Since utc=True was used, the dates are already in UTC.
    df.set_index('date', inplace=True)
    print("\nDataFrame Index is now set to Date (UTC).")
    print(df.head()) 
    
else:
    print("\nWarning: Date column not found. Skipping date conversion.")

In [ ]:
# --- 3. Duplicate and Consistency Checks ---

# Check for duplicate rows across all columns
duplicate_count = df.duplicated().sum()
print(f"\nTotal duplicate rows found: {duplicate_count}")

if duplicate_count > 0:
    # Drop duplicates, keeping the first instance
    df.drop_duplicates(inplace=True)
    print(f"Rows remaining after dropping duplicates: {df.shape[0]}")

# Ensure the 'headline' column is string type before analysis
df['headline'] = df['headline'].astype(str)

# Descriptive Statistics

In [67]:
# Calculate headline lengths
# df['headline_length'] = df['headline'].str.len()

# # Basic statistics
# headline_stats = df['headline_length'].describe()
# print("Headline Length Statistics:")
# print(headline_stats)
# import matplotlib.pyplot as plt
# print(plt.style.available)

# Visualize with PlotHelper
helper = PlotHelper()
helper.histogram(df, column='headline_length', bins=20, title='Distribution of Headline Lengths')

OSError: 'seaborn' is not a valid package style, path of style file, URL of style file, or library style name (library styles are listed in `style.available`)

In [ ]:
# Count articles per publisher
publisher_counts = df['publisher'].value_counts()
print("Number of Articles per Publisher:")
print(publisher_counts)

# Visualize top 10 with PlotHelper
top_publishers = publisher_counts.head(10).reset_index()
top_publishers.columns = ['publisher', 'count']
helper = PlotHelper()
helper.bar(top_publishers, x='publisher', height='count', title='Top 10 Most Active Publishers')

In [ ]:
# Ensure 'date' is datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Group by date and count articles
daily_counts = df.groupby(df['date'].dt.date).size().reset_index(name='count')
daily_counts['date'] = pd.to_datetime(daily_counts['date'])
print("Daily Article Counts:")
print(daily_counts.head())

# Plot trends over time with PlotHelper
helper = PlotHelper()
helper.line_plot(daily_counts, x='date', y='count', title='Article Frequency Over Time')

# Analyze by day of week
df['day_of_week'] = df['date'].dt.day_name()
weekly_counts = df['day_of_week'].value_counts().reset_index()
weekly_counts.columns = ['day', 'count']
print("Articles by Day of Week:")
print(weekly_counts)

# Visualize days with PlotHelper
helper.bar(weekly_counts, x='day', y='count', title='Articles by Day of the Week')

Text Analysis(Topic Modeling):